In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [55]:
import pandas as pd
import numpy as np
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot


In [166]:
df=pd.read_csv('/content/drive/MyDrive/train.csv')#,encoding='unicode_escape')
df.head()

,id,title,author,text,label,Unnamed: 5,Unnamed: 6
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,NaN,NaN
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,NaN,NaN
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,NaN,NaN
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,NaN,NaN
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,NaN,NaN


In [167]:
df.isnull().values.any()

True

In [168]:
df.isnull().values.sum()

44064

In [169]:
df.shape

(20800, 7)

In [170]:
df.tail(5)

,id,title,author,text,label,Unnamed: 5,Unnamed: 6
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,NaN,NaN
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,NaN,NaN
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,NaN,NaN
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,NaN,NaN
20799,20799,What Keeps the F-35 Alive,David Swanson,"David Swanson is an author, activist, journa...",1,NaN,NaN


In [171]:
df1=df.drop('Unnamed: 5',axis=1)
df1.shape

(20800, 6)

In [172]:
df1=df1.drop('Unnamed: 6', axis=1)
df1.shape

(20800, 5)

In [173]:
df2=df1.dropna()
df2.shape

(18284, 5)

In [174]:
df2.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [175]:
messages=df2["title"]
messages[0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [176]:
type(messages[0])

str

In [177]:
messages

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                        Why the Truth Might Get You Fired
3        15 Civilians Killed In Single US Airstrike Hav...
4        Iranian woman jailed for fictional unpublished...
                               ...                        
20795    Rapper T.I.: Trump a ’Poster Child For White S...
20796    N.F.L. Playoffs: Schedule, Matchups and Odds -...
20797    Macy’s Is Said to Receive Takeover Approach by...
20798    NATO, Russia To Hold Parallel Exercises In Bal...
20799                            What Keeps the F-35 Alive
Name: title, Length: 18284, dtype: object

In [178]:
import nltk
import re
from nltk.corpus import stopwords


In [179]:
y=df2["label"]
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: object

In [180]:
print(messages.head()," ",y.head())

0    House Dem Aide: We Didn’t Even See Comey’s Let...
1    FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                    Why the Truth Might Get You Fired
3    15 Civilians Killed In Single US Airstrike Hav...
4    Iranian woman jailed for fictional unpublished...
Name: title, dtype: object   0    1
1    0
2    1
3    1
4    1
Name: label, dtype: object


In [181]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [182]:
type(messages)

pandas.core.series.Series

In [183]:
for message in messages:
  print(message)

Streaming output truncated to the last 5000 lines.
Mitch McConnell, Paul Ryan Meet with Donald Trump Promise ‘Bold Agenda’ - Breitbart
Susan Rice, Ex-Obama Adviser, Is Back in Political Cross Hairs Over Surveillance - The New York Times
The Fix Is In: NBC Affiliate Accidentally Posts Election Results A Week Early: Hillary Wins Presidency 42% to Trump’s 40%
Original unreleased transcript of Sonia Gandhi’s interview with Rajdeep Sardesai obtained
Indian Man Accused in Multimillion-Dollar Call-Center Swindle Is Held - The New York Times
7 Teenage Deaths, but No Answers for Aboriginal Canadians - The New York Times
Fashioning Cast-Iron Pans for Today’s Cooks - The New York Times
Summer Sweat: Hollywood Braces for Possible Worst Box Office in Decade
Trump Executive Orders Target Trade Abuses Amid NAFTA Intrigue - Breitbart
New York Times’s Moscow Bureau Was Targeted by Hackers - The New York Times
Barbra Streisand Says Sexism Cost Her Multiple Oscar Nominations
Comey Investigation a Red Her

In [184]:
corpus=[]
for message in messages:
  message=re.sub('[^a-z,A-Z]',' ',message)
  corpus.append(message)

In [185]:
messages.shape

(18284,)

In [186]:
messages[18284]

'This Will END Hillary’s Campaign, HILLARY CLINTON IS GOING DOWN'

In [187]:
corpus

['House Dem Aide  We Didn t Even See Comey s Letter Until Jason Chaffetz Tweeted It',
 'FLYNN  Hillary Clinton, Big Woman on Campus   Breitbart',
 'Why the Truth Might Get You Fired',
 '   Civilians Killed In Single US Airstrike Have Been Identified',
 'Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery',
 'Jackie Mason  Hollywood Would Love Trump if He Bombed North Korea over Lack of Trans Bathrooms  Exclusive Video    Breitbart',
 'Beno t Hamon Wins French Socialist Party s Presidential Nomination   The New York Times',
 'A Back Channel Plan for Ukraine and Russia, Courtesy of Trump Associates   The New York Times',
 'Obama s Organizing for Action Partners with Soros Linked  Indivisible  to Disrupt Trump s Agenda',
 'BBC Comedy Sketch  Real Housewives of ISIS  Causes Outrage',
 'Russian Researchers Discover Secret Nazi Military Base  Treasure Hunter  in the Arctic  Photos ',
 'US Officials See No Link Between Trump and Russia',
 'Re  Yes, Th

In [188]:
for value in corpus:
  print(value)

Streaming output truncated to the last 5000 lines.
Mitch McConnell, Paul Ryan Meet with Donald Trump Promise  Bold Agenda    Breitbart
Susan Rice, Ex Obama Adviser, Is Back in Political Cross Hairs Over Surveillance   The New York Times
The Fix Is In  NBC Affiliate Accidentally Posts Election Results A Week Early  Hillary Wins Presidency     to Trump s    
Original unreleased transcript of Sonia Gandhi s interview with Rajdeep Sardesai obtained
Indian Man Accused in Multimillion Dollar Call Center Swindle Is Held   The New York Times
  Teenage Deaths, but No Answers for Aboriginal Canadians   The New York Times
Fashioning Cast Iron Pans for Today s Cooks   The New York Times
Summer Sweat  Hollywood Braces for Possible Worst Box Office in Decade
Trump Executive Orders Target Trade Abuses Amid NAFTA Intrigue   Breitbart
New York Times s Moscow Bureau Was Targeted by Hackers   The New York Times
Barbra Streisand Says Sexism Cost Her Multiple Oscar Nominations
Comey Investigation a Red Her

In [189]:
corpus1=[]
for value in corpus:
  value=value.lower()
  corpus1.append(value)

In [190]:
corpus1

['house dem aide  we didn t even see comey s letter until jason chaffetz tweeted it',
 'flynn  hillary clinton, big woman on campus   breitbart',
 'why the truth might get you fired',
 '   civilians killed in single us airstrike have been identified',
 'iranian woman jailed for fictional unpublished story about woman stoned to death for adultery',
 'jackie mason  hollywood would love trump if he bombed north korea over lack of trans bathrooms  exclusive video    breitbart',
 'beno t hamon wins french socialist party s presidential nomination   the new york times',
 'a back channel plan for ukraine and russia, courtesy of trump associates   the new york times',
 'obama s organizing for action partners with soros linked  indivisible  to disrupt trump s agenda',
 'bbc comedy sketch  real housewives of isis  causes outrage',
 'russian researchers discover secret nazi military base  treasure hunter  in the arctic  photos ',
 'us officials see no link between trump and russia',
 're  yes, th

In [191]:
from nltk.stem import PorterStemmer
ps=PorterStemmer()


In [192]:
corpus2=[]
for val in corpus1:
  val=val.split()
  print(val)
  val=[ps.stem(words) for words in val if not words in stopwords.words('english')]

  val=' '.join(val)
  corpus2.append(val)

Streaming output truncated to the last 5000 lines.
['mitch', 'mcconnell,', 'paul', 'ryan', 'meet', 'with', 'donald', 'trump', 'promise', 'bold', 'agenda', 'breitbart']
['susan', 'rice,', 'ex', 'obama', 'adviser,', 'is', 'back', 'in', 'political', 'cross', 'hairs', 'over', 'surveillance', 'the', 'new', 'york', 'times']
['the', 'fix', 'is', 'in', 'nbc', 'affiliate', 'accidentally', 'posts', 'election', 'results', 'a', 'week', 'early', 'hillary', 'wins', 'presidency', 'to', 'trump', 's']
['original', 'unreleased', 'transcript', 'of', 'sonia', 'gandhi', 's', 'interview', 'with', 'rajdeep', 'sardesai', 'obtained']
['indian', 'man', 'accused', 'in', 'multimillion', 'dollar', 'call', 'center', 'swindle', 'is', 'held', 'the', 'new', 'york', 'times']
['teenage', 'deaths,', 'but', 'no', 'answers', 'for', 'aboriginal', 'canadians', 'the', 'new', 'york', 'times']
['fashioning', 'cast', 'iron', 'pans', 'for', 'today', 's', 'cooks', 'the', 'new', 'york', 'times']
['summer', 'sweat', 'hollywood', 'br

In [193]:
corpus2

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton, big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia, courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'yes, paid govern troll social media, blogs, forum websit',
 'major leagu soccer, argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi clo

In [194]:
voc_size=5000
one_hot_rep=[one_hot(words, voc_size) for words in corpus2]
one_hot_rep

[[4063, 3367, 702, 3870, 549, 279, 497, 1952, 276, 1058],
 [384, 4380, 2238, 3083, 105, 46, 2779],
 [1467, 3854, 1720, 3115],
 [3676, 1402, 603, 3957, 4571, 2027],
 [3870, 105, 954, 2154, 4764, 3303, 105, 4827, 4946, 2177],
 [3171,
  985,
  365,
  4446,
  3738,
  384,
  4039,
  1859,
  4983,
  1787,
  1674,
  2309,
  2429,
  2179,
  2779],
 [3575, 3889, 456, 1916, 3100, 168, 1559, 3319, 2136, 4332, 1909],
 [3645, 1816, 620, 3867, 1718, 1757, 384, 2524, 2136, 4332, 1909],
 [3644, 1883, 3371, 1034, 1308, 548, 1746, 547, 384, 889],
 [877, 3224, 969, 783, 4971, 3211, 4425, 77],
 [841, 2898, 2760, 1733, 307, 4739, 4178, 4901, 329, 4283, 4598],
 [3957, 511, 549, 548, 384, 1718],
 [3078, 1305, 343, 2233, 4165, 3290, 3096, 4340, 394],
 [2427, 3401, 1897, 1265, 3517, 3667, 74, 2136, 4332, 1909],
 [3497, 325, 4998, 2384, 893, 2136, 4332, 1909],
 [1961, 3824, 934, 1931, 4733, 4693, 3526, 2057, 55, 4103],
 [1044, 4195, 4380],
 [4433, 4294, 934, 4731, 384, 3626, 98, 2779],
 [3419, 4339, 2238, 2054,

In [195]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
fix_length=20
embedded_string=pad_sequences(one_hot_rep, padding='pre', maxlen=fix_length)
print(embedded_string[0])

[   0    0    0    0    0    0    0    0    0    0 4063 3367  702 3870
  549  279  497 1952  276 1058]


In [196]:
#creating the LSTM model
n_features=50
model=Sequential()
model.add(Embedding(voc_size,n_features,input_length=fix_length))
model.add(LSTM(150))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 20, 50)            250000    
_________________________________________________________________
lstm_6 (LSTM)                (None, 150)               120600    
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 151       
Total params: 370,751
Trainable params: 370,751
Non-trainable params: 0
_________________________________________________________________


In [197]:
len(embedded_string), y.shape

(18284, (18284,))

In [198]:
print(type(embedded_string))

<class 'numpy.ndarray'>


In [199]:
print(type(y))

<class 'pandas.core.series.Series'>


In [200]:
new_X=np.array(embedded_string)
new_y=np.array(y)

In [201]:
print(new_X.shape)
print(new_y.shape)

(18284, 20)
(18284,)


In [202]:
print(new_X)

[[   0    0    0 ... 1952  276 1058]
 [   0    0    0 ...  105   46 2779]
 [   0    0    0 ... 3854 1720 3115]
 ...
 [   0    0    0 ... 2136 4332 1909]
 [   0    0    0 ... 2763 3131 3338]
 [   0    0    0 ...  142  908 4730]]


In [160]:
newdf=df2["label"]
newdf
# #newdf=re.sub('[^0-9]','0',newdf)

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18284, dtype: object

In [154]:
print(newdf.shape)

(18284,)


In [105]:
newdf=newdf.astype(int)
newdf

0        0
1        0
2        0
3        0
4        0
        ..
20795    0
20796    0
20797    0
20798    0
20799    0
Name: label, Length: 20800, dtype: int64

In [119]:
newdf.isnull().values.sum()

20800

In [120]:
newdf=newdf.dropna()
print(newdf.shape)

(0,)


In [115]:
newer_y=np.array(newdf)

In [107]:
print(type(newer_y[1234]))
print(type(newer_y))

<class 'numpy.int64'>
<class 'numpy.ndarray'>


In [84]:
print(type(new_X))
print(type(new_y))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [210]:
print(type(new_y[0]))
print(int(new_y[0])+int(new_y[1]))

<class 'str'>
1


In [221]:
sum=0
var=['1','0','a']
print(int(var[0]))
print(int(var[1]))
print(type(var[2])==str)
for i in range(len(var)):
  if (type(var[i])==str) and (var[i]!='0' or var[i]!='1'):
    var[i]=0
    print("@")
    print(var[i])
  else:
    print(int(var[i]))

1
0
True
@
0
@
0
@
0


In [224]:
var3=[]
var2=['1','0','a']
for i in range(len(var2)):
  if(var2[i]=='0' or var2[i]=='1'):
    var3.append(int(var2[i]))
  else:
    var3.append(0)
print(var3)

[1, 0, 0]


In [225]:
ay=[]

for i in range(len(new_y)):
  if(new_y[i]=='0' or new_y[i]=='1'):
    ay.append(int(new_y[i]))
  else:
    ay.append(0)
print(ay)

[1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 

In [228]:
anewy=np.array(ay)
print(type(anewy))

<class 'numpy.ndarray'>


In [229]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(new_X, anewy,train_size=0.30, random_state=101)

In [235]:
model.fit(X_train,y_train, validation_data=(X_test,y_test),epochs=50,batch_size=64)

Epoch 1/50
86/86 [==============================] - 7s 86ms/step - loss: 4.5548e-04 - accuracy: 1.0000 - val_loss: 0.6791 - val_accuracy: 0.8992
Epoch 2/50
86/86 [==============================] - 8s 89ms/step - loss: 3.9423e-04 - accuracy: 1.0000 - val_loss: 0.7130 - val_accuracy: 0.8973
Epoch 3/50
86/86 [==============================] - 7s 87ms/step - loss: 2.0520e-04 - accuracy: 1.0000 - val_loss: 0.7545 - val_accuracy: 0.8935
Epoch 4/50
86/86 [==============================] - 8s 96ms/step - loss: 1.8302e-04 - accuracy: 1.0000 - val_loss: 0.7703 - val_accuracy: 0.8952
Epoch 5/50
86/86 [==============================] - 8s 97ms/step - loss: 1.1504e-04 - accuracy: 1.0000 - val_loss: 0.8116 - val_accuracy: 0.8930
Epoch 6/50
86/86 [==============================] - 7s 85ms/step - loss: 1.0845e-04 - accuracy: 1.0000 - val_loss: 0.7989 - val_accuracy: 0.8951
Epoch 7/50
86/86 [==============================] - 7s 87ms/step - loss: 9.7436e-05 - accuracy: 1.0000 - val_loss: 0.8019 - val_ac

In [236]:
#predicting the model
y_pred=model.predict_classes(X_test)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [237]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,y_pred))

[[6665  632]
 [ 749 4753]]


In [234]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

0.8974138604578483
